In [ ]:
# !pip install langchain
# !pip install PyMuPDF
# !pip install openai
# !pip install chromadb
# !pip install tiktoken


In [ ]:
import os
# from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
# API Key OpenAI
os.environ["OPENAI_API_KEY"] = 'YOUR OPEN AI API KEY'

In [ ]:
persist_directory = "./storage"
# pdf_path = "for PDF File"

In [ ]:
# loader = PyMuPDFLoader(pdf_path) //For PDF file
# documents = loader.load()
loader = TextLoader("YOUR TXT FILE" , encoding='utf-8')
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=10)
texts = text_splitter.split_documents(documents)


In [ ]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
while True:
        user_input = input("Enter a query: ")
        print("Query: ", user_input)
        if user_input == "exit":
            print('It was a pleasure engaging in conversation with you. Thank you and take care.')
            break

        if not user_input:
            print("I am at your service, and I kindly await your query.")
            continue
             
        query = f"###Prompt {user_input}"
        try:
            llm_response = qa(query)
            print(llm_response["result"])
        
        except Exception as err:
            print('Exception occurred. Please try again', str(err))